# PySpark Tutorial - Dataframes
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

Up to now, we've see the RDD interface to PySpark. The RDD is a building block for more capable data structures such as the **dataframe** and **database**. These data structures are part of the [PySpark SQL library](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html) which, as the name implies, is influenced by standard SQL practices and queries.

The PySpark library has the **dataframe API**, but it does not support the **database API** -- that's only accessible via the Scala and Java libraries and through SQL queries.

The **database** is effectively an SQL relation -- i.e. rows and columns with a specific schema. The **dataframe** takes a little futher and constructs a labeled dataframe similar to the [Python Pandas](https://pandas.pydata.org/) interface or the [R dplyr](https://cran.r-project.org/web/packages/dplyr/vignettes/dplyr.html) interface for R.

In [35]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions
import numpy as np
import pandas as pd
import operator

We're going to use an airline information database as the example. You can download extended versions of the database [at this Dept. of Transportation website](https://www.transtats.bts.gov/DL_SelectFields.asp), but the data we're using is distributed with the course notes.

As with the RDD interface, we need a "context" to a remote machine. The [Spark SQL tutorial](https://spark.apache.org/docs/latest/sql-getting-started.html) has some information on this, but for complete information you need to look at the [Spark API documentation.](https://spark.apache.org/docs/latest/api/python/)

In this example, we're creating a local session (i.e. CPU's on JupyterHub).

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .master("local[*]")\
    .getOrCreate()

There are many ways to load data, including HDFS, a format called [Parquet](http://parquet.apache.org/), CSV files and so on. We'll use a compressed CSV file of the airline data.

In [3]:
flights = spark.read.load('airline-ontime-reporting.csv.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

The dataframe has a **schema** or type for each entry. All entries must have the same type or we'll see operations fail. In this example, we have asked that the schema be inferred -- this usually works, but if it doesn't we may need to take some extra steps (see below).

In [4]:
flights.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY_OF_MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- OP_UNIQUE_CARRIER: string (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
 |-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
 |-- DEST_CITY_MARKET_ID: integer (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY_NAME: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- ARR_TIME: integer (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- 

In [5]:
flights.columns

['YEAR',
 'MONTH',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'OP_UNIQUE_CARRIER',
 'ORIGIN_AIRPORT_ID',
 'ORIGIN_AIRPORT_SEQ_ID',
 'ORIGIN_CITY_MARKET_ID',
 'ORIGIN',
 'ORIGIN_CITY_NAME',
 'DEST_AIRPORT_ID',
 'DEST_AIRPORT_SEQ_ID',
 'DEST_CITY_MARKET_ID',
 'DEST',
 'DEST_CITY_NAME',
 'CRS_DEP_TIME',
 'DEP_DELAY',
 'ARR_TIME',
 'ARR_DELAY',
 'CANCELLED',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'LATE_AIRCRAFT_DELAY',
 '_c23']

In [6]:
print("There are", len(flights.columns), "columns and ", flights.count(), "rows")

There are 24 columns and  583985 rows


In [7]:
flights.dtypes

[('YEAR', 'int'),
 ('MONTH', 'int'),
 ('DAY_OF_MONTH', 'int'),
 ('DAY_OF_WEEK', 'int'),
 ('OP_UNIQUE_CARRIER', 'string'),
 ('ORIGIN_AIRPORT_ID', 'int'),
 ('ORIGIN_AIRPORT_SEQ_ID', 'int'),
 ('ORIGIN_CITY_MARKET_ID', 'int'),
 ('ORIGIN', 'string'),
 ('ORIGIN_CITY_NAME', 'string'),
 ('DEST_AIRPORT_ID', 'int'),
 ('DEST_AIRPORT_SEQ_ID', 'int'),
 ('DEST_CITY_MARKET_ID', 'int'),
 ('DEST', 'string'),
 ('DEST_CITY_NAME', 'string'),
 ('CRS_DEP_TIME', 'int'),
 ('DEP_DELAY', 'double'),
 ('ARR_TIME', 'int'),
 ('ARR_DELAY', 'double'),
 ('CANCELLED', 'double'),
 ('CARRIER_DELAY', 'double'),
 ('WEATHER_DELAY', 'double'),
 ('LATE_AIRCRAFT_DELAY', 'double'),
 ('_c23', 'string')]

The schema is inferred, but it can also be defined explicitly.

Note that one column is labeled `_c23`, which is showing up as "null". Perhaps this is bad data import?

Lets look at some of the values.

In [8]:
flights.show(5, truncate=False)

+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+----+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CARRIER_DELAY|WEATHER_DELAY|LATE_AIRCRAFT_DELAY|_c23|
+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+----+
|2019|1    |19          |6     

Let's pull out the values in one column -- the `select` method can be used to produce a new dataframe with just that column as an entry.

In [9]:
flights.select('_c23').show(5)

+----+
|_c23|
+----+
|null|
|null|
|null|
|null|
|null|
+----+
only showing top 5 rows



And we can slice out multiple columns, similar to Pandas. Again, this produces a new dataframe.

In [10]:
flights.select(['year', '_c23']).show(5)

+----+----+
|year|_c23|
+----+----+
|2019|null|
|2019|null|
|2019|null|
|2019|null|
|2019|null|
+----+----+
only showing top 5 rows



Alternatively, we can produce a
[Column object which has its own methods](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=column#pyspark.sql.Column). These are typically used in **column expressions** that produce indicies that can be used when selecting or filtering data.

For example, let's find all the rows where the mystery `_c23` column is not null.

In [11]:
flights.filter( flights._c23.isNotNull()).show(5)

+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+----+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CARRIER_DELAY|WEATHER_DELAY|LATE_AIRCRAFT_DELAY|_c23|
+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+----+
+----+-----+------------+------

Hmm.... This liooks like all the values are null. We could confirm this by selecting the column and looking at the distinct elements.

In [12]:
flights.select('_c23').distinct().show()

+----+
|_c23|
+----+
|null|
+----+



This this column is null, lets just drop it.

In [13]:
newFlights = flights.drop('_c23')

In [14]:
newFlights.show(5)

+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CARRIER_DELAY|WEATHER_DELAY|LATE_AIRCRAFT_DELAY|
+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+
|2019|    1|          19|          6|         

We often work with multiple columns of data in a dataframe. Some methods just use column names (corr, cov, crosstab, describe) and others can use column references, such as `newAir.ORIGIN`.

There are also a number of methods that work on columns or column expressions -- we've been using `select` already.

* `cube(*cols)`: column names (string) or column expressions or **both**.
* `drop(*cols)`: ***a list of column names OR a single column expression.***
* `groupBy(*cols)`: column name (string) or column expression or **both**.
* `rollup(*cols)`: column name (string) or column expression or **both**.
* `select(*cols)`: column name (string) or column expression or **both**.
* `sort(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `sortWithinPartitions(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `orderBy(*cols, **kwargs)`: column name (string) or column expression or **both**.
* `sampleBy(col, fractions, sed=None)`: a column name.
* `toDF(*cols)`: **a list of column names (string).**
* `withColumn(colName, col)`: `colName` refers to column name; `col` refers to a column expression.
* `withColumnRenamed(existing, new)`: takes column names as arguments.
* `filter(condition)`: ***condition** refers to a column expression that returns `types.BooleanType` of values. 

For example, we could determine the number of flights from a specific origin by group by the flight origin and then counting the entries in each group:

In [26]:
newFlights.groupBy(newFlights.ORIGIN).count().show(5)

+------+-----+
|ORIGIN|count|
+------+-----+
|   BGM|   61|
|   PSE|   65|
|   INL|   53|
|   MSY| 4597|
|   PPG|   11|
+------+-----+
only showing top 5 rows



If we want to find the files from a _specific_ origin airport, we can `filter` those entries:

In [27]:
newFlights.filter(newFlights.ORIGIN == 'DEN' ).show(5)

+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+---------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST| DEST_CITY_NAME|CRS_DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CARRIER_DELAY|WEATHER_DELAY|LATE_AIRCRAFT_DELAY|
+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+---------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+
|2019|    1|           1|          2|      

Let's say we want to find the number of flights that originate in Denver and go to other desinations. We can combine `filter` and `groupby` on the destination city.

In [45]:
newFlights.filter(newFlights.ORIGIN == 'DEN' )\
    .groupBy(newFlights.DEST_CITY_NAME)\
    .count().sort('count').show(5)

+--------------------+-----+
|      DEST_CITY_NAME|count|
+--------------------+-----+
|Pasco/Kennewick/R...|    2|
|          Albany, NY|    3|
|       Williston, ND|    5|
|     Little Rock, AR|    5|
|          Dayton, OH|    6|
+--------------------+-----+
only showing top 5 rows



We just named the column by specifying the column name (`count`) but we can also use the `col` function from the PySpark SQL library. This will also simplify setting the sort order by specific columns - for example, if we want to sort descending count but ascending on name, we would do:

In [43]:
from pyspark.sql.functions import col

In [49]:
newFlights.filter(newFlights.ORIGIN == 'DEN' )\
    .groupBy(newFlights.DEST_CITY_NAME)\
    .count().sort(col('count').asc(), col('DEST_CITY_NAME').asc()).show()

+--------------------+-----+
|      DEST_CITY_NAME|count|
+--------------------+-----+
|Pasco/Kennewick/R...|    2|
|          Albany, NY|    3|
|     Little Rock, AR|    5|
|       Williston, ND|    5|
|    Bend/Redmond, OR|    6|
|          Casper, WY|    6|
|          Dayton, OH|    6|
|         Buffalo, NY|    8|
|           Greer, SC|    8|
|Jackson/Vicksburg...|    8|
|       Lafayette, LA|    8|
|        Savannah, GA|    8|
|Harlingen/San Ben...|    9|
|       Knoxville, TN|    9|
|      Long Beach, CA|    9|
|      Huntsville, AL|   10|
|      Charleston, SC|   11|
|      Birmingham, AL|   12|
|       Pensacola, FL|   12|
|         Lincoln, NE|   19|
+--------------------+-----+
only showing top 20 rows



## Doing Joins

Again, everything boils down to a join in "big data". We can do joins between two dataframes much as in Pandas. Let's load a second dataframe that contains airline identifiers.

In [17]:
airlines = spark.read.load('unique-carriers.csv.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [18]:
airlines.show(5)

+----+--------------------+
|Code|         Description|
+----+--------------------+
| 02Q|       Titan Airways|
| 04Q|  Tradewind Aviation|
| 05Q| Comlux Aviation, AG|
| 06Q|Master Top Linhas...|
| 07Q| Flair Airlines Ltd.|
+----+--------------------+
only showing top 5 rows



Our flights data also has carrier information in the `OP_UNIQUE_CARRIER` column. Let's list out the distinct values by selecting that column, determining the distinct values and then showing it.

In [19]:
flights.select('OP_UNIQUE_CARRIER').distinct().show()

+-----------------+
|OP_UNIQUE_CARRIER|
+-----------------+
|               UA|
|               NK|
|               AA|
|               EV|
|               B6|
|               DL|
|               OO|
|               F9|
|               YV|
|               MQ|
|               OH|
|               HA|
|               G4|
|               YX|
|               AS|
|               WN|
|               9E|
+-----------------+



Now, let's join the airlines `Code`  with the flights `OP_UNIQUE_CARRIER`. This will result in data like the `flights` data but with two additional columns, `Code` (the join key) and `Description` (the full airline name).

In [20]:
flights.join(airlines, airlines.Code == flights.OP_UNIQUE_CARRIER).show(5)

+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-------------------+----+----+-----------------+
|YEAR|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CARRIER_DELAY|WEATHER_DELAY|LATE_AIRCRAFT_DELAY|_c23|Code|      Description|
+----+-----+------------+-----------+-----------------+-----------------+---------------------+---------------------+------+----------------+---------------+-------------------+-------------------+----+--------------+------------+---------+--------+---------+---------+-------------+-------------+-----------

From here, you could *e.g.* pull out all over the Denver to Chicago flights and list them by the airline name, *etc, etc*.

## Escape back into the world of RDD's

A dataframe is composed of `Row` objects and a dataframe (and database) is just a collection of those rows. You can pull out the row objects as RDD's and then operate on those, much as we did before.

In [21]:
flights.rdd.filter(lambda x: x['DEST'] == 'DEN').take(5)

[Row(YEAR=2019, MONTH=1, DAY_OF_MONTH=1, DAY_OF_WEEK=2, OP_UNIQUE_CARRIER='AA', ORIGIN_AIRPORT_ID=12892, ORIGIN_AIRPORT_SEQ_ID=1289208, ORIGIN_CITY_MARKET_ID=32575, ORIGIN='LAX', ORIGIN_CITY_NAME='Los Angeles, CA', DEST_AIRPORT_ID=11292, DEST_AIRPORT_SEQ_ID=1129202, DEST_CITY_MARKET_ID=30325, DEST='DEN', DEST_CITY_NAME='Denver, CO', CRS_DEP_TIME=1540, DEP_DELAY=-4.0, ARR_TIME=1915, ARR_DELAY=15.0, CANCELLED=0.0, CARRIER_DELAY=0.0, WEATHER_DELAY=0.0, LATE_AIRCRAFT_DELAY=0.0, _c23=None),
 Row(YEAR=2019, MONTH=1, DAY_OF_MONTH=2, DAY_OF_WEEK=3, OP_UNIQUE_CARRIER='AA', ORIGIN_AIRPORT_ID=12892, ORIGIN_AIRPORT_SEQ_ID=1289208, ORIGIN_CITY_MARKET_ID=32575, ORIGIN='LAX', ORIGIN_CITY_NAME='Los Angeles, CA', DEST_AIRPORT_ID=11292, DEST_AIRPORT_SEQ_ID=1129202, DEST_CITY_MARKET_ID=30325, DEST='DEN', DEST_CITY_NAME='Denver, CO', CRS_DEP_TIME=1540, DEP_DELAY=-3.0, ARR_TIME=1908, ARR_DELAY=8.0, CANCELLED=0.0, CARRIER_DELAY=None, WEATHER_DELAY=None, LATE_AIRCRAFT_DELAY=None, _c23=None),
 Row(YEAR=2019, 

Spark will attempt to interpret the types of the data but it's not always successful. By default, it will use the first 100 rows to determine the types. This may fail as indicated below:

In [50]:
onlyDen = spark.createDataFrame(flights.rdd.filter(lambda x: x['DEST'] == 'DEN'))

ValueError: Some of types cannot be determined by the first 100 rows, please try again with sampling

In this case, the solution is to sample the data randomly -- here we're going to sample 50% of the data to determine the types:

In [53]:
onlyDen = spark.createDataFrame(flights.rdd.filter(lambda x: x['DEST'] == 'DEN'), 
                                samplingRatio=0.5)

And again, the resulting data is a `Row` type:

In [54]:
onlyDen.take(3)

[Row(YEAR=2019, MONTH=1, DAY_OF_MONTH=1, DAY_OF_WEEK=2, OP_UNIQUE_CARRIER='AA', ORIGIN_AIRPORT_ID=12892, ORIGIN_AIRPORT_SEQ_ID=1289208, ORIGIN_CITY_MARKET_ID=32575, ORIGIN='LAX', ORIGIN_CITY_NAME='Los Angeles, CA', DEST_AIRPORT_ID=11292, DEST_AIRPORT_SEQ_ID=1129202, DEST_CITY_MARKET_ID=30325, DEST='DEN', DEST_CITY_NAME='Denver, CO', CRS_DEP_TIME=1540, DEP_DELAY=-4.0, ARR_TIME=1915, ARR_DELAY=15.0, CANCELLED=0.0, CARRIER_DELAY=0.0, WEATHER_DELAY=0.0, LATE_AIRCRAFT_DELAY=0.0, _c23=None),
 Row(YEAR=2019, MONTH=1, DAY_OF_MONTH=2, DAY_OF_WEEK=3, OP_UNIQUE_CARRIER='AA', ORIGIN_AIRPORT_ID=12892, ORIGIN_AIRPORT_SEQ_ID=1289208, ORIGIN_CITY_MARKET_ID=32575, ORIGIN='LAX', ORIGIN_CITY_NAME='Los Angeles, CA', DEST_AIRPORT_ID=11292, DEST_AIRPORT_SEQ_ID=1129202, DEST_CITY_MARKET_ID=30325, DEST='DEN', DEST_CITY_NAME='Denver, CO', CRS_DEP_TIME=1540, DEP_DELAY=-3.0, ARR_TIME=1908, ARR_DELAY=8.0, CANCELLED=0.0, CARRIER_DELAY=None, WEATHER_DELAY=None, LATE_AIRCRAFT_DELAY=None, _c23=None),
 Row(YEAR=2019, 

## Using SQL

It's clear that the Dataframe methods provide operations similar to those of SQL but in a more procedural or imperative form.

PySpark also has an SQL wrapper that lets us convert a `DataFrame` into an SQL relational table.

In [75]:
import pyspark.sql as sql

In [76]:
sqlContext = sql.SQLContext( spark.builder.getOrCreate() )

In [77]:
sqlContext.registerDataFrameAsTable(onlyDen, "onlyDen")

In [78]:
sqlContext.registerDataFrameAsTable(flights, "flights")

From there, we can do SQL queries and a query planner will construct the series of operations needed.

In [79]:
sqlContext.sql("SELECT COUNT(*) from onlyDEN").show(5)

+--------+
|count(1)|
+--------+
|   18498|
+--------+



In [81]:
sqlContext.sql("SELECT COUNT(*) from flights WHERE ORIGIN='DEN'").show(5)

+--------+
|count(1)|
+--------+
|   18507|
+--------+



In [86]:
sqlContext.sql("SELECT DEST_CITY_NAME, COUNT(*) as count from flights \
               WHERE ORIGIN='DEN' \
               GROUP BY DEST_CITY_NAME\
               ORDER BY count ASC, DEST_CITY_NAME ASC").show(10)

+--------------------+-----+
|      DEST_CITY_NAME|count|
+--------------------+-----+
|Pasco/Kennewick/R...|    2|
|          Albany, NY|    3|
|     Little Rock, AR|    5|
|       Williston, ND|    5|
|    Bend/Redmond, OR|    6|
|          Casper, WY|    6|
|          Dayton, OH|    6|
|         Buffalo, NY|    8|
|           Greer, SC|    8|
|Jackson/Vicksburg...|    8|
+--------------------+-----+
only showing top 10 rows

